In [33]:
import pandas as pd
import numpy as np
import string

import en_core_web_sm
import random
import requests
import language_tool_python
tool = language_tool_python.LanguageTool('en-US')

#language analysis tools
import nltk
nltk.download('punkt')
from nltk.corpus import cmudict
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.tokenize import word_tokenize, sent_tokenize

#web scraping tools
from bs4 import BeautifulSoup
import docx2txt

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\aguga\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\aguga\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [36]:
# EXTRACT POST TEXTS

#-------------- stackoverflow --------------
#https://stackoverflow.com/questions/49205608/how-can-i-extract-the-text-part-of-a-blog-page-while-web-scraping-using-beautifu
#https://stackoverflow.com/questions/69400419/how-to-get-other-column-value-with-the-first-non-null-value-of-a-column

#div class="question" and "answer"
stackPostOne = requests.get('https://stackoverflow.com/questions/49205608/how-can-i-extract-the-text-part-of-a-blog-page-while-web-scraping-using-beautifu')
soup = BeautifulSoup(stackPostOne.text, 'lxml')
stackpostOne_text = soup.find('div', class_='question').text + soup.find('div', class_='answer').text
#stackpostOne_text = File_object.write(soup.find('div', class_='question').text + soup.find('div', class_='answer').text)
#print(stackpostOne_text)

stackPostTwo = requests.get('https://stackoverflow.com/questions/69400419/how-to-get-other-column-value-with-the-first-non-null-value-of-a-column')
soup = BeautifulSoup(stackPostTwo.text, 'lxml')
stackpostTwo_text = soup.find('div', class_='question').text #+ soup.find('div', class_='answer').text
#print(stackpostTwo_text)

#-------------- hardware zone --------------
#https://www.hardwarezone.com.sg/review-apple-ipad-mini-6th-generation-2021-review-singapore-price-specs
#https://www.hardwarezone.com.sg/tech-news-usb-c-new-logos-usb4-40gbps-240w

#div class = content resizeable
hardPostOne = requests.get('https://www.hardwarezone.com.sg/review-apple-ipad-mini-6th-generation-2021-review-singapore-price-specs')
soup = BeautifulSoup(hardPostOne.text, 'lxml')
hardPostOne_text = soup.find('div', class_='content resizeable').text 
#print(hardPostOne_text)

hardPostTwo = requests.get('https://www.hardwarezone.com.sg/tech-news-usb-c-new-logos-usb4-40gbps-240w')
soup = BeautifulSoup(hardPostTwo.text, 'lxml')
hardPostTwo_text = soup.find('div', class_='content resizeable').text 
#print(hardPostTwo_text)

#-------------- channelnews asia  (blocked webscraping after 1st run of code) --------------
#https://www.channelnewsasia.com/sport/torino-tougher-task-chelsea-allegri-warns-juventus-2215766
#https://www.channelnewsasia.com/asia/seller-basks-squid-game-fame-his-sweet-and-deadly-treat-2215296

#div class = content
# cnaOne = requests.get('https://www.channelnewsasia.com/sport/torino-tougher-task-chelsea-allegri-warns-juventus-2215766')
# soup = BeautifulSoup(cnaOne.text, 'lxml')
# cnaOne_text = soup.find('div', class_='content').text 

# cnaTwo = requests.get('https://www.channelnewsasia.com/asia/seller-basks-squid-game-fame-his-sweet-and-deadly-treat-2215296')
# soup = BeautifulSoup(cnaTwo.text, 'lxml')
# cnaTwo_text = soup.find('div', class_='content').text 

cnaOne_text = docx2txt.process("cnaOne.docx")
#print(cnaOne_text)
cnaTwo_text = docx2txt.process("cnaTwo.docx")
#print(cnaTwo_text)




In [43]:
################################## ANALYSE WRITING STYLE ###############################################

#-------------- average word count per sentence --------------
def Avg_SentLengthByWord(text):
    tokens = sent_tokenize(text)
    avg_sent = np.average([len(token.split()) for token in tokens])
    return avg_sent

#-------------- average word length --------------
def Avg_wordLength(str):
    str.translate(string.punctuation)
    tokens = word_tokenize(str, language='english')
    st = [",", ".", "'", "!", '"', "#", "$", "%", "&", "(", ")", "*", "+", "-", ".", "/", ":", ";", "<", "=", '>', "?",
          "@", "[", "\\", "]", "^", "_", '`', "{", "|", "}", '~', '\t', '\n']
    stop = stopwords.words('english') + st
    words = [word for word in tokens if word not in stop]
    avg_word = np.average([len(word) for word in words])
    return avg_word

#-------------- check good grammar --------------
def grammar_check(text):
    matches = tool.check(text)
    error_count = len(matches)
    #print(matches)
    return error_count

#-------------- check capitalisation of sentence --------------
def cap_check(text):
    error_cap = 0
    for sentence in sent_tokenize(text):
        if not sentence[0].isupper(): # sentence does not start with a capital letter
            error_cap+=1
    return error_cap


######################################## RESULTS ######################################################


#-------------- average word count per sentence --------------
average_sent = (Avg_SentLengthByWord(stackpostOne_text)+Avg_SentLengthByWord(stackpostTwo_text))/2
print('Average word count per sentence for Stackoverflow:' ,str.format('{0:.3f}', average_sent) )
average_sent = (Avg_SentLengthByWord(hardPostOne_text)+Avg_SentLengthByWord(hardPostTwo_text))/2
print('Average word count per sentence for HardwareZone:' ,str.format('{0:.3f}', average_sent) )
average_sent = (Avg_SentLengthByWord(cnaOne_text)+Avg_SentLengthByWord(cnaTwo_text))/2
print('Average word count per sentence for CNA:' ,str.format('{0:.3f}', average_sent) )

#-------------- average word length --------------
average = (Avg_wordLength(stackpostOne_text)+Avg_wordLength(stackpostTwo_text))/2
print('Average word length for Stackoverflow:' ,str.format('{0:.3f}', average) )
average = (Avg_wordLength(hardPostOne_text)+Avg_wordLength(hardPostTwo_text))/2
print('Average word length for HardwareZone:' ,str.format('{0:.3f}', average) )
average = (Avg_SentLengthByWord(cnaOne_text)+Avg_SentLengthByWord(cnaTwo_text))/2
print('Average word length for CNA:' ,str.format('{0:.3f}', average) )

#-------------- check good grammar --------------
average = (grammar_check(stackpostOne_text)+grammar_check(stackpostTwo_text))/2
print('Grammar Check Score for Stackoverflow:' ,str.format('{0:.3f}', average) )
average = (grammar_check(hardPostOne_text)+grammar_check(hardPostTwo_text))/2
print('Grammar Check Score for HardwareZone:' ,str.format('{0:.3f}', average) )
average = (grammar_check(cnaOne_text)+grammar_check(cnaTwo_text))/2
print('Grammar Check Score for CNA:' ,str.format('{0:.3f}', average) )

#-------------- check capitalisation of sentence --------------
average = (cap_check(stackpostOne_text)+cap_check(stackpostTwo_text))/2
print('Capitalisation Check Score for Stackoverflow:' ,str.format('{0:.3f}', average) )
average = (cap_check(hardPostOne_text)+cap_check(hardPostTwo_text))/2
print('Capitalisation Check Score for HardwareZone:' ,str.format('{0:.3f}', average) )
average = (cap_check(cnaOne_text)+cap_check(cnaTwo_text))/2
print('Capitalisation Check Score for CNA:' ,str.format('{0:.3f}', average) )

Average word count per sentence for Stackoverflow: 28.071
Average word count per sentence for HardwareZone: 19.861
Average word count per sentence for CNA: 24.476
Average word length for Stackoverflow: 5.336
Average word length for HardwareZone: 5.846
Average word length for CNA: 24.476
Grammar Check Score for Stackoverflow: 65.500
Grammar Check Score for HardwareZone: 15.000
Grammar Check Score for CNA: 25.500
Capitalisation Check Score for Stackoverflow: 3.000
Capitalisation Check Score for HardwareZone: 2.000
Capitalisation Check Score for CNA: 5.500
